In [1]:
from pathlib import Path
import pandas as pd
import country_converter as coco
import plotly.express as px

In [48]:
data_path = r"..\data\salaries.csv"
df = pd.read_csv(data_path)
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,300000,USD,300000,US,0,US,M
1,2022,SE,FT,Data Engineer,225000,USD,225000,US,0,US,M
2,2022,SE,FT,Data Scientist,198440,USD,198440,US,100,US,M
3,2022,SE,FT,Data Scientist,144000,USD,144000,US,100,US,M
4,2022,SE,FT,Applied Scientist,191475,USD,191475,US,100,US,M
...,...,...,...,...,...,...,...,...,...,...,...
1090,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
1091,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
1092,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
1093,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [3]:
df.sample(n=5, random_state=42)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
533,2022,MI,FT,Data Engineer,80000,EUR,84711,ES,100,ES,M
139,2022,SE,FT,Data Engineer,105000,USD,105000,US,100,US,M
88,2022,SE,FT,Data Scientist,191475,USD,191475,US,100,US,M
841,2021,MI,FT,Research Scientist,53000,EUR,62649,FR,50,FR,M
985,2021,SE,FT,Machine Learning Engineer,1799997,INR,24342,IN,100,IN,L


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           1095 non-null   int64 
 1   experience_level    1095 non-null   object
 2   employment_type     1095 non-null   object
 3   job_title           1095 non-null   object
 4   salary              1095 non-null   int64 
 5   salary_currency     1095 non-null   object
 6   salary_in_usd       1095 non-null   int64 
 7   employee_residence  1095 non-null   object
 8   remote_ratio        1095 non-null   int64 
 9   company_location    1095 non-null   object
 10  company_size        1095 non-null   object
dtypes: int64(4), object(7)
memory usage: 94.2+ KB


In [8]:
dropped_df = df.drop(columns=["salary", "salary_currency"])
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           1095 non-null   int64 
 1   experience_level    1095 non-null   object
 2   employment_type     1095 non-null   object
 3   job_title           1095 non-null   object
 4   salary_in_usd       1095 non-null   int64 
 5   employee_residence  1095 non-null   object
 6   remote_ratio        1095 non-null   int64 
 7   company_location    1095 non-null   object
 8   company_size        1095 non-null   object
dtypes: int64(3), object(6)
memory usage: 77.1+ KB


In [49]:
df["working_overseas"] = df.employee_residence != df.company_location

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           1095 non-null   int64 
 1   experience_level    1095 non-null   object
 2   employment_type     1095 non-null   object
 3   job_title           1095 non-null   object
 4   salary              1095 non-null   int64 
 5   salary_currency     1095 non-null   object
 6   salary_in_usd       1095 non-null   int64 
 7   employee_residence  1095 non-null   object
 8   remote_ratio        1095 non-null   int64 
 9   company_location    1095 non-null   object
 10  company_size        1095 non-null   object
 11  working_overseas    1095 non-null   bool  
dtypes: bool(1), int64(4), object(7)
memory usage: 95.3+ KB


In [50]:
df.remote_ratio.value_counts()

100    687
0      280
50     128
Name: remote_ratio, dtype: int64

In [41]:
[print(c[0], c[1].value_counts().size) for c in df.items()]

work_year 3
experience_level 4
employment_type 4
job_title 59
salary 385
salary_currency 18
salary_in_usd 511
employee_residence 64
remote_ratio 3
company_location 59
company_size 3
working_overseas 2


[None, None, None, None, None, None, None, None, None, None, None, None]

In [59]:
converter = coco.CountryConverter()

In [79]:
df.replace(
    {
        "remote_ratio": {
            0: "No remote work",
            50: "Partially remote",
            100: "Fully remote",
        },
        "experience_level": {
            "EN": "Junior",
            "MI": "Intermediate",
            "SE": "Expert",
            "EX": "Director",
        },
        "employment_type": {
            "PT": "Part-time",
            "FT": "Full-time",
            "CT": "Contract",
            "FL": "Freelance",
        },
        "employee_residence": dict(
            zip(
                df.employee_residence,
                coco.convert(df.employee_residence, to="name_short"),
            )
        ),
        "company_location": dict(
            zip(df.company_location, coco.convert(df.company_location, to="name_short"))
        ),
        "company_size": {"S": "Small", "M": "Medium", "L": "Large"},
    },
)
# dict(zip(df.company_location, coco.convert(df.company_location, to='name_short')))

# df.employee_residence.replace()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,working_overseas
0,2022,Expert,Full-time,Data Engineer,300000,USD,300000,United States,No remote work,United States,Medium,False
1,2022,Expert,Full-time,Data Engineer,225000,USD,225000,United States,No remote work,United States,Medium,False
2,2022,Expert,Full-time,Data Scientist,198440,USD,198440,United States,Fully remote,United States,Medium,False
3,2022,Expert,Full-time,Data Scientist,144000,USD,144000,United States,Fully remote,United States,Medium,False
4,2022,Expert,Full-time,Applied Scientist,191475,USD,191475,United States,Fully remote,United States,Medium,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2020,Expert,Full-time,Data Scientist,412000,USD,412000,United States,Fully remote,United States,Large,False
1091,2021,Intermediate,Full-time,Principal Data Scientist,151000,USD,151000,United States,Fully remote,United States,Large,False
1092,2020,Junior,Full-time,Data Scientist,105000,USD,105000,United States,Fully remote,United States,Small,False
1093,2020,Junior,Contract,Business Data Analyst,100000,USD,100000,United States,Fully remote,United States,Large,False


In [45]:
(
    df.astype(
        {
            "work_year": "category",
            "experience_level": "category",
            "employment_type": "category",
            "company_size": "category",
        }
    )
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           1095 non-null   category
 1   experience_level    1095 non-null   category
 2   employment_type     1095 non-null   category
 3   job_title           1095 non-null   object  
 4   salary              1095 non-null   int64   
 5   salary_currency     1095 non-null   object  
 6   salary_in_usd       1095 non-null   int64   
 7   employee_residence  1095 non-null   object  
 8   remote_ratio        1095 non-null   int64   
 9   company_location    1095 non-null   object  
 10  company_size        1095 non-null   category
 11  working_overseas    1095 non-null   bool    
dtypes: bool(1), category(4), int64(3), object(4)
memory usage: 66.0+ KB


In [20]:
df.employee_residence = coco.convert(names=df.employee_residence, to="ISO3")
df.company_location = coco.convert(names=df.company_location, to="ISO3")

In [26]:
print(df.employee_residence)
print(df.employee_residence.value_counts())

0       USA
1       USA
2       USA
3       USA
4       USA
       ... 
1090    USA
1091    USA
1092    USA
1093    USA
1094    IND
Name: employee_residence, Length: 1095, dtype: object
USA    711
GBR     74
IND     42
CAN     33
DEU     31
      ... 
IRL      1
THA      1
IDN      1
EGY      1
MLT      1
Name: employee_residence, Length: 64, dtype: int64


In [84]:
import pandas as pd
import country_converter as coco


csv_path = r".\data\salaries.csv"

df = pd.read_csv(csv_path).astype(
    {
        "work_year": "category",
        "experience_level": "category",
        "employment_type": "category",
        "company_size": "category",
        "remote_ratio": "category",
    }
)


df = df.drop(columns=["salary", "salary_currency"])

df["working_overseas"] = df.employee_residence != df.company_location

df = df.replace(
    {
        "remote_ratio": {
            "0": "No remote work",
            "50": "Partially remote",
            "100": "Fully remote",
        },
        "experience_level": {
            "EN": "Junior",
            "MI": "Intermediate",
            "SE": "Expert",
            "EX": "Director",
        },
        "employment_type": {
            "PT": "Part-time",
            "FT": "Full-time",
            "CT": "Contract",
            "FL": "Freelance",
        },
        "company_size": {
            "S": "Small",
            "M": "Medium",
            "L": "Large",
        },
        "employee_residence": dict(
            zip(
                df.employee_residence,
                coco.convert(df.employee_residence, to="name_short"),
            )
        ),
        "company_location": dict(
            zip(
                df.company_location,
                coco.convert(df.company_location, to="name_short"),
            )
        ),
    },
)
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           1095 non-null   category
 1   experience_level    1095 non-null   category
 2   employment_type     1095 non-null   category
 3   job_title           1095 non-null   object  
 4   salary_in_usd       1095 non-null   int64   
 5   employee_residence  1095 non-null   object  
 6   remote_ratio        1095 non-null   category
 7   company_location    1095 non-null   object  
 8   company_size        1095 non-null   category
 9   working_overseas    1095 non-null   bool    
dtypes: bool(1), category(5), int64(1), object(3)
memory usage: 41.5+ KB
None


,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,working_overseas
0,2022,Expert,Full-time,Data Engineer,300000,United States,0,United States,Medium,False
1,2022,Expert,Full-time,Data Engineer,225000,United States,0,United States,Medium,False
2,2022,Expert,Full-time,Data Scientist,198440,United States,100,United States,Medium,False
3,2022,Expert,Full-time,Data Scientist,144000,United States,100,United States,Medium,False
4,2022,Expert,Full-time,Applied Scientist,191475,United States,100,United States,Medium,False
...,...,...,...,...,...,...,...,...,...,...
1090,2020,Expert,Full-time,Data Scientist,412000,United States,100,United States,Large,False
1091,2021,Intermediate,Full-time,Principal Data Scientist,151000,United States,100,United States,Large,False
1092,2020,Junior,Full-time,Data Scientist,105000,United States,100,United States,Small,False
1093,2020,Junior,Contract,Business Data Analyst,100000,United States,100,United States,Large,False


In [92]:
print(df.employee_residence.unique())
print(df.company_location.unique())
set(df.employee_residence.unique()) - set(df.company_location.unique())

['United States' 'Spain' 'Germany' 'Nigeria' 'Portugal' 'Czech Republic'
 'Australia' 'Costa Rica' 'Canada' 'Turkey' 'Chile' 'Puerto Rico'
 'United Kingdom' 'India' 'Finland' 'France' 'Thailand' 'Brazil'
 'Netherlands' 'Greece' 'Denmark' 'Bolivia' 'Philippines' 'Aland Islands'
 'Dominican Republic' 'Argentina' 'Belgium' 'Austria' 'Singapore' 'Egypt'
 'Indonesia' 'Italy' 'United Arab Emirates' 'Ireland' 'Slovenia'
 'Malaysia' 'Japan' 'Estonia' 'Pakistan' 'Poland' 'Honduras' 'Tunisia'
 'Switzerland' 'Russia' 'Algeria' 'Vietnam' 'Iraq' 'Ukraine' 'Bulgaria'
 'Hungary' 'Mexico' 'Jersey' 'Serbia' 'Kenya' 'Colombia' 'New Zealand'
 'Iran' 'Romania' 'China' 'Hong Kong' 'Moldova' 'Luxembourg' 'Croatia'
 'Malta']
['United States' 'Spain' 'Germany' 'Nigeria' 'Portugal' 'Czech Republic'
 'Australia' 'Canada' 'Turkey' 'Puerto Rico' 'United Kingdom' 'India'
 'Finland' 'American Samoa' 'France' 'Thailand' 'Brazil' 'Netherlands'
 'Greece' 'Denmark' 'Bolivia' 'Philippines' 'Albania' 'Argentina'
 'Belgiu

{'Aland Islands',
 'Bulgaria',
 'Costa Rica',
 'Dominican Republic',
 'Hong Kong',
 'Jersey',
 'Serbia',
 'Tunisia'}